# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [64]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

# # Google developer API key
# from config import g_key

# # Configure gmaps
# gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [49]:
# File to Load 
city_data = "cities.csv"

# Read the Data and print dataframe
city_data_df = pd.read_csv("cities.csv")
city_data_df.dropna()
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [50]:
# Store latitude and longitude in locations
locations = city_data_df[["Lat", "Lng"]]

humidity = city_data_df["Humidity"].astype(int)

In [51]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [74]:
narrowed_city_df= city_data_df.loc[(city_data_df["Cloudiness"] < 7) &
                                  (city_data_df["Humidity"] <= 50)&
                                  (city_data_df["Max Temp"] >= 70) &
                                  (city_data_df["Wind Speed"] <= 10)]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
151,151,coahuayana,1,MX,1558378783,22,18.62,-100.35,100.50,4.38
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
306,306,chunhuhub,5,MX,1558378846,35,19.59,-88.59,96.72,7.02
416,416,kirkuk,0,IQ,1558378874,16,35.47,44.40,86.82,7.36
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17
510,510,atar,1,MR,1558378892,12,20.52,-13.05,95.00,9.17
518,518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72
525,525,mecca,1,SA,1558378872,48,21.43,39.83,88.62,1.74


In [75]:
narrowed_city_df.reset_index()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
1,142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
2,151,151,coahuayana,1,MX,1558378783,22,18.62,-100.35,100.50,4.38
3,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
4,306,306,chunhuhub,5,MX,1558378846,35,19.59,-88.59,96.72,7.02
5,416,416,kirkuk,0,IQ,1558378874,16,35.47,44.40,86.82,7.36
6,444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17
7,510,510,atar,1,MR,1558378892,12,20.52,-13.05,95.00,9.17
8,518,518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72
9,525,525,mecca,1,SA,1558378872,48,21.43,39.83,88.62,1.74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
# Setup column to hold hotel info
narrowed_city_df["Hotel Name"]=""
narrowed_city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08,
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36,


In [78]:
# Find the closest hotel to each location
target_search = "Hotel"
target_radius = 5000

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

lat = response["results"][0]["geometry"]["location"]["lat"]
lng = response["results"][0]["geometry"]["location"]["lng"]
print(response.url)

target_coordinates = f"{'lat'}{'lng'}"


# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}




TypeError: 'Response' object is not subscriptable

In [65]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 27.1301495,
                    "lng": 81.6995822
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.13149757989272,
                        "lng": 81.70094067989272
                    },
                    "southwest": {
                        "lat": 27.12879792010727,
                        "lng": 81.69824102010729
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "b93e3cf261c7615e094e90a1f27d40d4c1c6e048",
            "name": "Sareef Baba Juma Biryani Wale",
            "place_id": "ChIJcU3jqA-QmTkRO9QltLy-wCs",
            "plus_code": {
                "compound_code": "4MJX+3R Sakraura, Uttar Pradesh, India",
                "global_code": "7MV34MJX+3R"
         

In [66]:
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

Sareef Baba Juma Biryani Wale
Sadar Bazar, Colonelganj


In [70]:
hotel_df = []
for city in narrowed_city_df:
    response = requests.get(base_url, params = params).json()
    hotel_df.append(places_data["results"][0]["name"])
    try:
        narrowed_city_df.loc['Hotel Name'] = hotel_df
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
narrowed_city_df.head()

ValueError: cannot set a row with mismatched columns

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]



NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map

# Display Map